### Hourly Specific Features

1. **Hour of the Day**: Differentiate between peak hours (like morning and evening commute times) and off-peak hours.

2. **Weather Conditions**: 
   - **Temperature**: Hourly temperature variations.
   - **Precipitation Intensity**: Sudden rain or snow can deter riders.
   - **Wind Speed and Direction**: Can affect riding comfort and safety.

3. **Air Quality Index**: Can change hourly, affecting outdoor activity levels.

4. **Traffic Congestion**: Real-time traffic data to indicate road conditions.

5. **Public Transport Usage**: Fluctuations in public transport ridership could correlate with bike usage.

6. **Local Events**: Events happening in or around the city at specific hours that could increase or decrease bike usage.

### Daily Specific Features

1. **Day of the Week**: Differentiating between weekdays and weekends.

2. **Sunrise/Sunset Times**: Influences riding habits, especially for leisure riders.

3. **Special Daily Events**: Local daily events, like markets or sports games.

4. **School and Work Schedules**: School days vs. holidays, typical workday patterns.

5. **Day-Specific Promotions**: Any bike-share or cycling promotions specific to certain days.

6. **Public Transport Schedules**: Variation in public transport services (e.g., reduced service on weekends).

### Weekly Specific Features

1. **Week of the Year**: Capture seasonal trends and holiday periods.

2. **Weekly Weather Forecast**: Anticipated weather conditions for the week.

3. **City-Wide Weekly Events**: Larger events that span multiple days or occur weekly.

4. **Cycling Trends and Campaigns**: Any week-long campaigns or trends promoting cycling.

5. **Weekly Traffic Patterns**: Changes in road traffic conditions.

6. **Tourist Influx**: If the city is a tourist destination, weekly variations in tourist numbers can affect bike rides.


In [1]:
# EDA 

# https://www.kaggle.com/code/abhishekmamidi/time-series-preprocessing-to-modelling

In [2]:
# TODO 

# Strike
# Public Transport Accessibility
# Socio economics
# Bike lanes/ Traffic 
# Residential vs Business area

# OneHotEncode also cyclic date data
# Outliers
# Scaling (revert the weather data scaler, and apply RollingStandardScaler)
# Check for general upward trend (stationary)
# Denoising using fft_denoiser (should be done for every feature i guess)

# Use previous data (there seems to be more available on open_source)

# Split based on forecasting, not only random

In [3]:
from pathlib import Path
import problem

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from jours_feries_france.compute import JoursFeries
from jours_feries_france.compute import JoursFeries
import datetime


from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
def fft_denoiser(x, n_components, to_real=True):

    n = len(x)
    
    # compute the fft
    fft = np.fft.fft(x, n)
    
    # compute power spectrum density
    # squared magnitud of each fft coefficient
    PSD = fft * np.conj(fft) / n
    
    # keep high frequencies
    _mask = PSD > n_components
    fft = _mask * fft
    
    # inverse fourier transform
    clean_data = np.fft.ifft(fft)
    
    if to_real:
        clean_data = clean_data.real
    
    return clean_data

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class RollingStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self, window, mode='rolling'):
        self.window = window
        self.mode = mode
        
        # to fill in code
        self.pd_object = None
        self.w_mean = None
        self.w_std = None
        self.__fitted__ = False
        
    def __repr__(self):
        return f"RollingStandardScaler(window={self.window}, mode={self.mode})"
        
    def fit(self, X, y=None):
        self.pd_object = getattr(X, self.mode)(self.window)
        self.w_mean = self.pd_object.mean()
        self.w_std = self.pd_object.std()
        self.__fitted__ = True
        
        return self
    
    def transform(self, X):
        self._check_fitted()
        
        standardized = X.copy()
        return (standardized - self.w_mean) / self.w_std
    
    def inverse_transform(self, X):
        self._check_fitted()
        
        unstandardized = X.copy()
        return  (unstandardized * self.w_std) + self.w_mean
        
        
    def _check_fitted(self):
        """ Checks if the algorithm is fitted. """
        if not self.__fitted__:
            raise ValueError("Please, fit the algorithm first.")
            
            


In [6]:
def basic_filter(data, mode='rolling', window=30, threshold=3):
    
    msg = f"Type must be of pandas.Series but {type(data)} was passed."
    assert isinstance(data, pd.Series), msg
    
    series = data.copy()
    
    # rolling/expanding objects
    pd_object = getattr(series, mode)(window=window)
    mean = pd_object.mean()
    std = pd_object.std()
    
    upper_bound = mean + threshold * std
    lower_bound = mean - threshold * std
    
    outliers = ~series.between(lower_bound, upper_bound)
    # fill false positives with False
    outliers.iloc[:window] = [False] * window
    
    series = series.to_frame()
    series['outliers'] = np.array(outliers.astype('int').values)
    series.columns = ['Close', 'Outliers']
    
    return series

In [7]:

def cyclic_time(X, col, n):
        
    unique_values = X[col].unique()
    cos_unique = np.cos(unique_values / n * 2 * np.pi)
    sin_unique = np.sin(unique_values / n * 2 * np.pi)
        
    return unique_values, cos_unique, sin_unique

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "week"] = X["date"].dt.isocalendar().week.astype(np.int64)
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour
    

    # Create a binary column for workdays: Workday if it's a weekday (Mon-Fri) and not a holiday
    X.loc[:, "is_workday"] = (X['weekday'] < 5).astype(int)
    
    
    X["date"] = X['date'].dt.date
    
    years = list(range(min(X['year']), max(X['year']) + 1))
    
    bank_holidays = []
    for year in years: 
        bank_holidays += JoursFeries.for_year(year).values()
        
        
    school_holidays = pd.read_csv('data/school_holidays.csv')
    school_holidays['date'] = pd.to_datetime(school_holidays['date'])

    school_holidays = school_holidays.loc[(school_holidays['date'].dt.date >= min(X['date'])) & (school_holidays['date'].dt.date <= max(X['date']))]
    school_holidays = school_holidays.loc[school_holidays['vacances_zone_c'] == True].date.dt.date.values

    X.loc[:, "if_School_Holiday"] = X['date'].isin(school_holidays).astype(int)
    X.loc[:, "is_Bank_Holiday"] = X['date'].isin(bank_holidays).astype(int)

    X.loc[:, "is_Rush_Hour"]= 0
    X.loc[((X['hour'] >= 6) & (X['hour'] <= 8)) | ((X['hour'] >= 17) & (X['hour'] <= 20)), 'is_Rush_Hour'] = 1

    
    zipped = zip(['hour', 'weekday', 'day', 'week', 'month'], [24, 7, 31, 52, 12])

    for col, n in zipped:
        unique_values, cos_unique, sin_unique = cyclic_time(X, col, n)
        X[f'{col}_cos'] = X[col].replace(unique_values, sin_unique)
        X[f'{col}_sin'] = X[col].replace(unique_values, cos_unique)

        
    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=['date', 'hour', 'weekday', 'day', 'week', 'month'])


In [8]:
def _additional_features(X):
    X = X.copy()  # modify a copy of X
    
    X['date'] = pd.to_datetime(X['date'])
    
    
    weather_data = scaled_weather_data.loc[(scaled_weather_data['datetime'] >= min(X.date)) & (scaled_weather_data['datetime'] <= max(X.date))]

    merged_data = pd.merge(X, weather_data,  how='left', left_on=['date'], right_on=['datetime'], validate="m:1")
    merged_data = merged_data.drop(columns=['datetime'])
    
    merged_data = pd.merge(merged_data, velib_stats,  how='left', right_on=['site_id'], left_on=['site_id'])
    
    final_data = _encode_dates(merged_data)
    
    return final_data

In [9]:
scaled_weather_data = pd.read_csv('data/scaled_weather_data.csv')
scaled_weather_data['datetime'] = pd.to_datetime(scaled_weather_data['datetime'])

In [10]:
velib_stats = pd.read_csv('data/velib_processed.csv')


In [11]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [12]:
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585


In [13]:
from statsmodels.tsa.stattools import adfuller


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [14]:
# https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html
stationary_test = data.set_index('date')['log_bike_count']
adf_test(stationary_test.sample(n=50000, random_state=1))


Results of Dickey-Fuller Test:
Test Statistic                  -157.112951
p-value                            0.000000
#Lags Used                         1.000000
Number of Observations Used    49998.000000
Critical Value (1%)               -3.430481
Critical Value (5%)               -2.861598
Critical Value (10%)              -2.566801
dtype: float64


In [16]:
outliers = basic_filter(data['log_bike_count'])
outliers.loc[outliers['Outliers'] != 0]

,Close,Outliers
53148,2.564949,1
57868,2.484907,1
57893,3.828641,1
66181,0.693147,1
66273,0.000000,1
...,...,...
919731,0.000000,1
921014,0.693147,1
925239,0.000000,1
928156,0.000000,1


In [17]:
transformed_data = _additional_features(data)

transformed_data.drop(columns=['counter_id', 'site_id', 'bike_count', 'counter_installation_date', 'coordinates', 'counter_technical_id'], inplace=True)

one_hot_encoded_data = pd.get_dummies(transformed_data, columns=['counter_name', 'site_name'], dtype=int) 

In [18]:
y = one_hot_encoded_data['log_bike_count']
X = one_hot_encoded_data.drop(columns=['log_bike_count'])

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 18)

# Various hyper-parameters to tune
xgb1 = XGBRegressor()

"""parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}
              
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
        
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}
           
           

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)"""



xgb1 = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.07, max_depth = 7, min_child_weight = 4, n_estimators = 500, nthread = 4, objective = 'reg:linear', subsample = 0.7)
xgb1.fit(X_train, y_train)

[13:55:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: 

In [ ]:
xgb1.score(X_train, y_train)

In [ ]:
xgb1.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, xgb1.predict(X_train), squared=False)

In [ ]:
final_test = pd.read_parquet(Path("data") / "test_final.parquet")

In [ ]:
final_transformed_data = _additional_features(final_test)
final_transformed_data.drop(columns=['counter_id', 'site_id', 'counter_installation_date', 'coordinates', 'counter_technical_id'], inplace=True)
final_one_hot_encoded_data = pd.get_dummies(final_transformed_data, columns=['counter_name', 'site_name'], dtype=int) 



In [ ]:
final_predictions = xgb1.predict(final_one_hot_encoded_data)

In [ ]:
final_submission = pd.DataFrame({'log_bike_count': final_predictions, 'Id': final_one_hot_encoded_data.index}, columns=['Id', 'log_bike_count'])
final_submission.to_csv('final_submission.csv', index=False)

In [ ]:
final_submission

In [ ]:
final_submission.shape

In [ ]:
pd.read_csv('data/sample_submission.csv')

In [ ]:
mean_squared_error(y_test, xgb1.predict(X_test), squared=False)

In [ ]:
#merged_data[merged_data.temp.isna()].isna().sum()

# All the columns where name.isna() lack weather data. data set incomplete. (2021-03-28 02:00:00). Imputed from data before
# merged_data.sort_values(by='date').loc[merged_data.name.isna()].date.unique()



## Linear model

Let's now construct our first linear model with [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). We use a few helper functions defined in `problem.py` of the starting kit to load the public train and test data:

In [ ]:
from sklearn.preprocessing import FunctionTransformer

data_transformer = FunctionTransformer(_additional_features, validate=False)
data_transformer.fit_transform(data).head()

In [ ]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

In [ ]:
X_train.head(2)

and

In [ ]:
y_train

Where `y` contains the `log_bike_count` variable. 

The test set is in the future as compared to the train set,

In [ ]:
print(
    f'Train: n_samples={X_train.shape[0]},  {X_train["date"].min()} to {X_train["date"].max()}'
)
print(
    f'Test: n_samples={X_test.shape[0]},  {X_test["date"].min()} to {X_test["date"].max()}'
)

In [ ]:
transformed_data.info()


In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(transformed_data[['counter_name', 'site_name']])
onehot_encoded = enc.transform(transformed_data[['counter_name', 'site_name']]).toarray()

In [ ]:
onehot_encoded = pd.get_dummies(transformed_data[['counter_name', 'site_name']], dtype=float)

In [ ]:
final_data = pd.merge(transformed_data, onehot_encoded, left_index=True, right_index=True).drop(columns=['counter_name','site_name'])



In [ ]:
final_data.isna().sum()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

data_transformer = FunctionTransformer(_additional_features)
date_cols = ['year', 'hour_cos', 'hour_sin', 'weekday_cos', 'weekday_sin', 'day_cos', 
             'day_sin', 'week_cos', 'week_sin', 'month_cos', 'month_sin']

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name"]

preprocessor = ColumnTransformer(
    [
        ("date", OneHotEncoder(handle_unknown="ignore"), date_cols),
        ("cat", categorical_encoder, categorical_cols),
    ]
)

regressor = Ridge()

pipe = make_pipeline(data_transformer, preprocessor, regressor)
pipe.fit(X_train, y_train)

We then evaluate this model with the RMSE metric,

In [ ]:
from sklearn.metrics import mean_squared_error

print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, pipe.predict(X_test), squared=False):.2f}"
)

The model doesn't have enough capacity to generalize on the train set, since we have lots of data with relatively few parameters. However it happened to work somewhat better on the test set. We can compare these results with the baseline predicting the mean value,

In [ ]:
print("Baseline mean prediction.")
print(
    f"Train set, RMSE={mean_squared_error(y_train, np.full(y_train.shape, y_train.mean()), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, np.full(y_test.shape, y_test.mean()), squared=False):.2f}"
)

which illustrates that we are performing better than the baseline.

Let's visualize the predictions for one of the stations,

In [ ]:
mask = (
    (X_test["counter_name"] == "Totem 73 boulevard de Sébastopol S-N")
    & (X_test["date"] > pd.to_datetime("2021/09/01"))
    & (X_test["date"] < pd.to_datetime("2021/09/02"))
)

df_viz = X_test.loc[mask].copy()
df_viz["bike_count"] = np.exp(y_test[mask.values]) - 1
df_viz["bike_count (predicted)"] = np.exp(pipe.predict(X_test[mask])) - 1

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

df_viz.plot(x="date", y="bike_count", ax=ax)
df_viz.plot(x="date", y="bike_count (predicted)", ax=ax, ls="--")
ax.set_title("Predictions with Ridge")
ax.set_ylabel("bike_count")

In [ ]:
# School Holidays
#!pip install vacances-scolaires-france

# Bank Holidays
#!pip install jours-feries-france==0.5.1

# Trash

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoded_data = pd.get_dummies(extract_final, columns = ['counter_name', 'site_name', 'preciptype'], dtype=int) 


In [ ]:

scaler = StandardScaler()
scaler.fit(one_hot_encoded_data)
scaler.transform(one_hot_encoded_data)

y = one_hot_encoded_data['log_bike_count']
X = one_hot_encoded_data.drop(columns=['log_bike_count'])

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 18)

# Various hyper-parameters to tune
xgb1 = XGBRegressor()

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

xgb1 = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.07, max_depth = 7, min_child_weight = 4, n_estimators = 500, nthread = 4, objective = 'reg:linear', silent = 1, subsample = 0.7)
xgb1.fit(X_train, y_train)

In [ ]:
xgb1.score(x_train, y_train)

In [ ]:
xgb1.score(x_test, y_test)

In [ ]:
from sklearn.linear_model import Ridge
regressor = Ridge()

regressor.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, regressor.predict(X_train))

In [ ]:
mean_squared_error(y_test, regressor.predict(X_test))

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_train, regressor.predict(X_train))